<a href="https://colab.research.google.com/github/jaredsloan45/2023-GES-Cleaning/blob/main/GES_cleaning_user_2022_2023_with_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Survey Data Cleaning**

---



**To start:**

---



Download chrome extension or chrome ap: Open in Colab (program that allows you to write and execute Python in your browser it has nothing to do with the CBGLCollab). Only need to do this once.

Download data from qualtrics: File must be in csv not excel and "use numeric values." Don't modify or delete any rows.

If necessary, re-code any programs in qualtrics to match code book.

Press the first "play" button below. Allow colab to access your drive when prompted and then select "choose files." Open the desired file.






In [ ]:
import numpy as np
import pandas as pd
import natsort as ns
import re
from google.colab import files
from google.colab import drive
import io
drive.mount('drive',force_remount=True)

uploaded = files.upload()

Mounted at drive


Saving GES_Fall22_Summer23_Oct11_23_Export.csv to GES_Fall22_Summer23_Oct11_23_Export (2).csv


5.   In the box below enter the:*actions for 22-23 cycle

*a) file name exactly as written (including spaces) btw single quotations w .csv at the end (ie: '2021-2022 GES.csv')

b) Redundant_program_cols (this is for insitutions that have 2 questions for program code ie term number and then the program. For 22-23 cycle this is CFHI and WPI). If need to add in the future, add name of the column that you want to delete). Don't need to do anything for 22-23 cycle.

c) multi_answer_qs (name of the column as qualtrics geneates it, number of possible answer choices). If needed can adjust in the future. Don't need to do anything for 22-23 cycle.  

*d) starting unique id number into the appropriate variables below.

e) open_questions_pre/post: the name of the columns with any qualitative questions (e.g. 'Q14'). To update, either delete old column names or add new ones in quotes, separating each column with a comma. This includes any qualitative institution-specific questions. Don't need to do anything for the 22-23 cycle.

f) useful_metadata: the name of any columns containing metadata that we want to include in the final files (e.g. name, intitution, program, etc.). The formatting for this is the same as the open_questions_pre/post. Don't need to do anything for the 22-23 cycle.
*6.   Scroll to the bottom of the code (#export files to drive) and enter the google drive folder where you want your cleaned data to go.
7.   Check the "GES data cleaning code documentation" to see if anything else needs to be updated. Don't need to do anything for the 22-23 cycle.
*8.   Press the second "play" button to run the rest of the code, the cleaned files should appear in the desired folder after about a minute



In [ ]:
from types import prepare_class

#1:User inputted variables
file_name = 'GES_Fall22_Summer23_Oct11_23_Export (2).csv'
redundant_program_cols = ['CFHI','WPI']
multi_answer_qs = [ ['PF42a',7],['PF43',10]]
id_start = 12200
open_questions_pre = ['Q14', 'Q15', 'Q17', 'Q18', 'Q27', 'Q36', 'Q45', 'Q46', 'Q54', 'Q55', 'Q64', 'Q71', 'Q82', 'Q84_4_TEXT', 'Q85_8_TEXT', 'Q86_2_TEXT', 'Q87_2_TEXT', 'Q98_11_12_TEXT', 'ElWAD1', 'ElWAD2', 'ELWAD3', 'ElWAD4', 'ElWAD5', 'ElH1', 'ElH2', 'ElH3', 'WPI1a']
open_questions_post = [ 'Q14P', 'Q15P', 'Q17P', 'Q18P', 'Q27P', 'Q36P', 'Q45P', 'Q46P', 'Q54P', 'Q55P', 'Q64P', 'Q71P', 'Q82P', 'Q137P', 'Q139P', 'Q141P', 'Q143P', 'Q149P', 'Q151P', 'Q153P', 'PF33_10_TEXT', 'PF34_3_TEXT', 'PF35_6_TEXT', 'PF36_7_TEXT', 'PF41_1_TEXT','ELWAD1P', 'ElWAD2P', 'ELWAD3P', 'ELWAD4P', 'ElH1P', 'ElH2P', 'ElH3P', 'ElH4P', 'ElH5P', 'WPI1aP', 'WPI2P', 'WPI3P', 'WPI4P', 'Have2P']
useful_metadata = ['Q1_1','Q1_2','Q1_3','UniqueID', 'Institution','Program','Pre_Post_x','Pre_Post_y']
#read file into dataframe
df1 = pd.read_csv(io.BytesIO(uploaded[file_name]))

question_text = df1.iloc[:1,:]
question_text = question_text[open_questions_pre + open_questions_post]
df1 = df1.iloc[2:,:]

#2:Initial formatting changes:

#fixing the institution column name - Qualtrics generates it with extra characters
df1 = df1.rename(
    columns={
        'Institution\xa0':'Institution',
      })

#Combine all program columns into one
#first delete redundant columns (e.g. the place-based/virtual/other question for CFHI)
df1 = df1.drop(df1.loc[:,redundant_program_cols],axis = 1)

#Then combine remaining columns -- take all cols betweeen "institution" and "email" and merge them, each row should only have a value in one column
program_start = df1.columns.get_loc('Institution')+1
program_end = df1.columns.get_loc('Email')
df1 = df1.rename(
    columns = {
        df1.columns[program_start]: "Program",
    }
)
for i in range(program_start,program_end):
  if df1.columns[i] not in redundant_program_cols:
    df1['Program'] = df1['Program'].combine_first(df1.iloc[:,i])
df1 = df1.drop(df1.iloc[:,program_start+1:program_end],axis=1)

#Formatting emails and middle initial/name -- make emails lowercase and trim whitespace to serve as a unique identifier, removes middle initials entered as "None" or "N/A"
df1['Email'] = df1['Email'].str.lower()
df1['Email'] = df1['Email'].str.strip()
df1['Q1_2'] = df1['Q1_2'].str.title().str.strip()
df1.loc[(df1['Q1_2']=='None')|(df1['Q1_2']=='N/A'),'Q1_2'] = pd.Series(dtype=str)

#remove rows without values in Q6
df1 = df1[pd.notna(df1['Q6'])|pd.notna(df1['Q6P'])]
print(df1[pd.isna(df1['Program'])])


                StartDate              EndDate Status        IPAddress  \
2645  2023-06-06 14:43:28  2023-06-06 14:45:49      0  129.170.194.147   

     Progress Duration (in seconds) Finished         RecordedDate  \
2645      100                   141        1  2023-06-06 14:45:50   

             ResponseId RecipientLastName  ...   CRP   CR GCRP  GCR HRBP  HRB  \
2645  R_2e2Ol04TtlnofEJ               NaN  ...  3.25  NaN  4.0  NaN  2.5  NaN   

        ODP   OD  PVP   PV  
2645  3.125  NaN  3.5  NaN  

[1 rows x 268 columns]


<ipython-input-23-9a4f336068e2>:12: DtypeWarning: Columns (2,4,5,6,9,10,11,12,13,14,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,42,43,44,45,46,47,48,49,50,53,56,57,58,59,60,61,62,63,65,66,67,68,69,70,71,72,74,75,76,77,78,79,82,83,84,85,88,89,90,91,92,93,94,96,97,98,99,100,101,103,104,105,106,107,108,109,110,111,112,114,115,119,121,123,124,125,126,127,128,129,130,131,132,133,134,135,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,159,162,163,164,165,166,167,168,169,171,172,173,174,175,176,177,178,180,181,182,183,184,185,188,189,190,191,194,195,196,197,198,199,200,202,203,204,205,206,207,209,210,211,212,213,214,215,216,217,218,220,222,229,231,233,235,237,239,240,241,242,244,247,248,249,250,251,252,253,254,259,260,261,262,263,264,269,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(io.BytesIO(uploaded[file_name]))


In [ ]:
from types import prepare_class

#1:User inputted variables
file_name = 'GES_Fall22_Summer23_Oct11_23_Export (1).csv'
redundant_program_cols = ['CFHI','WPI']
multi_answer_qs = [ ['PF42a',7],['PF43',10]]
id_start = 12200
open_questions_pre = ['Q14', 'Q15', 'Q17', 'Q18', 'Q27', 'Q36', 'Q45', 'Q46', 'Q54', 'Q55', 'Q64', 'Q71', 'Q82', 'Q84_4_TEXT', 'Q85_8_TEXT', 'Q86_2_TEXT', 'Q87_2_TEXT', 'Q98_11_12_TEXT', 'ElWAD1', 'ElWAD2', 'ELWAD3', 'ElWAD4', 'ElWAD5', 'ElH1', 'ElH2', 'ElH3', 'WPI1a']
open_questions_post = [ 'Q14P', 'Q15P', 'Q17P', 'Q18P', 'Q27P', 'Q36P', 'Q45P', 'Q46P', 'Q54P', 'Q55P', 'Q64P', 'Q71P', 'Q82P', 'Q137P', 'Q139P', 'Q141P', 'Q143P', 'Q149P', 'Q151P', 'Q153P', 'PF33_10_TEXT', 'PF34_3_TEXT', 'PF35_6_TEXT', 'PF36_7_TEXT', 'PF41_1_TEXT','ELWAD1P', 'ElWAD2P', 'ELWAD3P', 'ELWAD4P', 'ElH1P', 'ElH2P', 'ElH3P', 'ElH4P', 'ElH5P', 'WPI1aP', 'WPI2P', 'WPI3P', 'WPI4P', 'Have2P']
useful_metadata = ['Q1_1','Q1_2','Q1_3','UniqueID', 'Institution','Program','Pre_Post_x','Pre_Post_y']
#read file into dataframe
df1 = pd.read_csv(io.BytesIO(uploaded[file_name]))

question_text = df1.iloc[:1,:]
question_text = question_text[open_questions_pre + open_questions_post]
df1 = df1.iloc[2:,:]

#2:Initial formatting changes:

#fixing the institution column name - Qualtrics generates it with extra characters
df1 = df1.rename(
    columns={
        'Institution\xa0':'Institution',
      })

#Combine all program columns into one
#first delete redundant columns (e.g. the place-based/virtual/other question for CFHI)
df1 = df1.drop(df1.loc[:,redundant_program_cols],axis = 1)

#Then combine remaining columns -- take all cols betweeen "institution" and "email" and merge them, each row should only have a value in one column
program_start = df1.columns.get_loc('Institution')+1
program_end = df1.columns.get_loc('Email')
df1 = df1.rename(
    columns = {
        df1.columns[program_start]: "Program",
    }
)
for i in range(program_start,program_end):
  if df1.columns[i] not in redundant_program_cols:
    df1['Program'] = df1['Program'].combine_first(df1.iloc[:,i])
df1 = df1.drop(df1.iloc[:,program_start+1:program_end],axis=1)

#Formatting emails and middle initial/name -- make emails lowercase and trim whitespace to serve as a unique identifier, removes middle initials entered as "None" or "N/A"
df1['Email'] = df1['Email'].str.lower()
df1['Email'] = df1['Email'].str.strip()
df1['Q1_2'] = df1['Q1_2'].str.title().str.strip()
df1.loc[(df1['Q1_2']=='None')|(df1['Q1_2']=='N/A'),'Q1_2'] = pd.Series(dtype=str)

#remove rows without values in Q6
df1 = df1[pd.notna(df1['Q6'])|pd.notna(df1['Q6P'])]

#3:Drop duplicates

#sort by progress and duration
df = df1.sort_values(by=['Progress','Duration (in seconds)'],axis=0,ascending=[False,False])
#delete unneeded metadata columns
df = df.drop(['Status','IPAddress'],axis=1).drop(df.loc[:,'Duration (in seconds)':'UserLanguage'],axis=1)
#drop duplicates -- because of the sorting, we are able to simply keep the first record of any duplicates
df = df.drop_duplicates(subset=['Email','Program','Pre_Post'],keep='first')

df = df[pd.notna(df['Program']) & pd.notna(df['Pre_Post'])]

#ensure pre_post column stored as numbers rather than text
df['Pre_Post']=df['Pre_Post'].astype(np.int32)
df['Program']=df['Program'].astype(np.int32)
#4:Unique IDs
#create UniqueID column
df.insert(df.columns.get_loc('Q1_3')+1,'UniqueID',"")

#Generate unique ids for pre
unique_ids = range(id_start,id_start+len(df[df['Pre_Post']<2]))
df.loc[df['Pre_Post']<2,'UniqueID']=unique_ids

#Generate unique ids for post

#create subset of the pre data containing just emails, program # and UniqueIDs
emails_pre = df.loc[df['Pre_Post']<2,['Email','Program','UniqueID']]
emails_pre = emails_pre.rename(
    columns ={
        'UniqueID':'id'
    })

#split pre and post data
df_pre = df[df['Pre_Post']<2]
df_post1 = df[df['Pre_Post']>1]

#Assigns unique ids for post-survey matched cases
df_post = pd.merge(df_post1, emails_pre, on =['Email','Program'], how ='left')

df_post['UniqueID']=df_post['id']
#removes redundant column
df_post = df_post.drop('id',axis=1)
#generate ids for unmatched post starting at 1+the max of the unique ids in the pre-survey
post_ids = range(int(max(df_pre['UniqueID']))+1,len(df_post[pd.isna(df_post['UniqueID'])])+int(max(df_pre['UniqueID']))+1)
df_post.loc[pd.isna(df_post['UniqueID']),'UniqueID']=post_ids

#5:Begin building output files

#create researcher file
researcher = df_post[['UniqueID','Q1_1','Q1_2','Q1_3', 'Institution', 'Program', 'Res1','Res2']]

#remove post questions from pre-survey and pre questions from post-survey
df_pre_cleaned = df_pre.drop(df_pre.loc[:,'Q6P':],axis=1,inplace=False)
index1 = df_post.columns.get_loc('Q6P')
index2 = df_post.columns.get_loc('Q6')
df_post_cleaned = df_post.drop(df_post.iloc[:,index2:index1],axis=1,inplace=False).drop(df_post.loc[:,['Res1','Res2']],axis=1,inplace=False).drop(df_post.loc[:,'CAP':],axis=1,inplace=False)

#6:Adjust demographic and program factor data

#For any students selecting multiple racial categories, re-assigns column to "8" for "other"
df_pre_cleaned.loc[df_pre_cleaned['Q85'].str.len()>1,'Q85']=8
#renames Q95_1 and Q95_2 into Q95 and Q96 respectively
df_pre_cleaned.rename(
    columns = {
        'Q95_1':'Q95',
        'Q95_2':'Q96'
    },inplace=True)


#Split any columns with multiple possible answers into separate columns for each choice -- 1 for yes, empty for no
for array in multi_answer_qs:
  cols_add = []
  col_loc = df_post_cleaned.columns.get_loc(array[0])
  for i in range(array[1],0,-1):
    col_name = array[0] + '_' + str(i)
    df_post_cleaned.insert(col_loc,col_name,"")
    cols_add.append([i,col_name])
    df_post_cleaned.loc[df_post_cleaned[array[0]].str.find(str(i))>-1,col_name]=1
  df_post_cleaned = df_post_cleaned.drop(array[0],axis=1)

#7:Generate final outputs

# Split into quant and qual based on user-inputted lists at top
pre_open_q = []
q_names_pre = []
for array in open_questions_pre:
  pre_open_q.append(array[0])
  q_names_pre.append(array[1])
post_open_q = []
q_names_post = []
for array in open_questions_post:
  post_open_q.append(array[0])
  q_names_post.append(array[1])

pre_quant = df_pre_cleaned.drop(open_questions_pre,axis=1)
post_quant = df_post_cleaned.drop(open_questions_post,axis=1)


#Create matched column for quant -- for the pre data, check each case to see if it exists in the post, assign value to "Matched" column accordingly, same for post.
quant_dbs = [pre_quant,post_quant]
for i in range(2):
  quant_dbs[i].insert(quant_dbs[i].columns.get_loc('Program')+1,'Matched',"")
  quant_dbs[i].loc[quant_dbs[i]['UniqueID'].isin(quant_dbs[(i+1)%2]['UniqueID']),'Matched'] =3
  quant_dbs[i].loc[~(quant_dbs[i]['UniqueID'].isin(quant_dbs[(i+1)%2]['UniqueID'])),'Matched'] = i+1

#9:Merging qual responses

#recreate the full dataset (qual and quant, pre and post)
df_full = pd.merge(df_pre_cleaned.loc[:,'Q1_1':],df_post_cleaned.loc[:,'UniqueID':],on=['UniqueID','Institution','Program'],how='outer')

#Add the qual question text into the dataset
df_qual_combined = pd.concat([question_text, df_full], ignore_index=True)
#Filter full dataset to include only qual questions and the necessary metadata
df_qual = df_qual_combined[useful_metadata + open_questions_pre + open_questions_post]

#reorder qual columns

#The idea here is to search the column names for certain letters that are indicative of different questions -- we can identify the program factors because they start with "PF", the demographic questions because they start with "Q" and end with "TEXT", and the institution specific questions as the only non-metadata columns that don't start with "Q" or "PF".
qual_cols = df_qual.columns[8:]
qual_qs=[]
demog_qs = []
qual_program_factors = []
qual_info = []
inst_spec_qual = []
#need to make sure _TEXT ending is specific to demographic qs (not including program factors or institution-specific qs)
for col in qual_cols:
  if re.search('^Q',col) != None:
    if re.search('TEXT$',col) != None:
      demog_qs.append(col)
    else:
      qual_qs.append(col)
  elif re.search('^PF',col) != None:
    qual_program_factors.append(col)
  else:
    inst_spec_qual.append(col)
#Take the separate sets of column names and sort them to put, for example, "Q14P" directly after "Q14". The ns.natsorted function accomplishes this, since a standard sort would place "Q120" before "Q15".
sorted_qs = ns.natsorted(qual_qs)
sorted_pf = ns.natsorted(qual_program_factors)
sorted_is = ns.natsorted(inst_spec_qual,alg=ns.IGNORECASE)
sorted_ds = ns.natsorted(demog_qs)

#Generate a new Pre_Post column to reflect the "matched" quant column (1 for pre only, 2 for post only, 3 for both)
df_qual['Pre_Post']=df_qual['Pre_Post_x'].fillna(0)+df_qual['Pre_Post_y'].fillna(0)
df_qual_final = df_qual[['Q1_1','Q1_2','Q1_3','UniqueID','Institution','Program','Pre_Post'] +  sorted_qs + sorted_ds + sorted_pf + sorted_is]

#export files to drive -- edit text after "drive/My Drive/" to match your drive folder. Enter name exactly as is including spaces between double quotations. (ie "drive/My Drive/GESDataCleaningCode")

pre_quant.to_csv('Pre_Closed.csv')
!cp Pre_Closed.csv "drive/My Drive/GES Cleaned"

post_quant.to_csv('Post_Closed.csv')
!cp Post_Closed.csv "drive/My Drive/GES Cleaned"

df_qual_final.to_csv('Qualitative.csv')
!cp Qualitative.csv "drive/My Drive/GES Cleaned"

researcher.to_csv('Researcher.csv')
!cp Researcher.csv "drive/My Drive/GES Cleaned"



KeyError: ignored